In [8]:
import os
import cv2
import gc
from skimage import color, data, restoration
import cv2
import numpy as np
from skimage.restoration import estimate_sigma
from skimage.filters import median
import config
import imutils
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

def weiner_noise_reduction(img):
    # data.astronaut()
    img = color.rgb2gray(img)
    from scipy.signal import convolve2d
    psf = np.ones((5, 5)) / 25
    img = convolve2d(img, psf, 'same')
    img += 0.1 * img.std() * np.random.standard_normal(img.shape)
    deconvolved_img = restoration.wiener(img, psf, 1100)

    return deconvolved_img


def estimate_noise(img):
    # img = cv2.imread(image_path)
    return estimate_sigma(img, multichannel=True, average_sigmas=True)


def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    enoise = estimate_noise(image)
    noise_free_image = weiner_noise_reduction(image)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    fingerprint = gray - noise_free_image
    fingerprint = fingerprint / 255
    filtered_img = median(fingerprint, selem=None, out=None, mask=None, shift_x=False,
                          shift_y=False, mode='nearest', cval=0.0, behavior='rank')
    colored = cv2.cvtColor(filtered_img, cv2.COLOR_GRAY2BGR)
    # print('-----------------')
    # cv2.imshow('filtered_image', filtered_img)
    # colored = cv2.cvtColor(filtered_img, cv2.COLOR_GRAY2BGR)
    # print(colored)
    # cv2.imshow('colored', colored)
    return colored



In [9]:
from keras import  Sequential, optimizers, applications
from keras.layers import  Dropout, Dense, Flatten
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import config


In [10]:

img_height = 256
img_width = 384
# build the VGG16 network
model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
# add the model on top of the convolutional base


new_model = Sequential() #new model
for layer in model.layers:
    new_model.add(layer)

new_model.add(top_model) # now this works

print('Model loaded.')

print(new_model.summary())


# model_aug.load_weights('k64 binary 25percent stride8/fine_tuned_model_resnet_64_adam_weights.h5')
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
new_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

'''TODO change path of weights here...'''
new_model.load_weights('models/casia-I/new_model.h5')

Model loaded.
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 256, 384, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 384, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 192, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 192, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 192, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 96, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 64, 

In [11]:
def infer_on_image(imagepath):
    cv_image = cv2.imread(imagepath)
    plt.imshow(imutils.resize(cv_image, width=400))
    plt.show()
    print('processing...')
    cv_image = cv2.resize(cv_image, (384,256))
    processed_image = preprocess_image(cv_image)
    y_pred = new_model.predict_classes(np.array([processed_image]))
    return processed_image, y_pred

In [12]:

def test_inference_on_dataset():
    folder_path = os.path.join('casia-dataset', 'CASIA1')
    authentic_path = os.path.join(folder_path, 'Au')
    authenticImages = os.listdir(authentic_path)
    print("Prediction on the Authentic Dataset")
    for index in range(len(authenticImages)):
        if index == 5: break
        imagepath = os.path.join(authentic_path, authenticImages[index])
        processed_image, prediction = infer_on_image(imagepath)
        print(imagepath)
        if prediction == 0:
            print("Result=> Authentic")
        else:
            print("Result=> Forged")
    
    authentic_path = os.path.join(folder_path, 'Sp')
    authenticImages = os.listdir(authentic_path)
    
    print("")
    print("")
    print("")
    print("Prediction on the Forged Dataset")

    
    for index in range(len(authenticImages)):
        if index == 5: break
        imagepath = os.path.join(authentic_path, authenticImages[index])
        processed_image, prediction = infer_on_image(imagepath)
        print(imagepath)
        if prediction == 0:
            print("Prediction=> Authentic")
        else:
            print("Prediction=> Forged")

        

In [ ]:
test_inference_on_dataset()